In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Saskylakh,RU,71.9167,114.0833,26.53,97,99,6.89,overcast clouds
1,1,Kaitangata,NZ,-46.2817,169.8464,45.84,92,99,15.82,overcast clouds
2,2,Airai,TL,-8.9266,125.4092,57.72,90,32,3.62,scattered clouds
3,3,Midyat,TR,37.4247,41.3393,61.56,32,25,7.38,scattered clouds
4,4,Burgeo,CA,47.6167,-57.6152,52.36,67,22,7.61,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
11,11,Kapaa,US,22.0752,-159.3190,75.18,99,75,4.61,broken clouds
12,12,Ukiah,US,39.1502,-123.2078,77.43,42,0,1.01,clear sky
15,15,Bengkulu,ID,-3.8004,102.2655,77.18,86,97,2.68,overcast clouds
18,18,Hilo,US,19.7297,-155.0900,88.29,82,100,4.61,overcast clouds
20,20,Albany,US,42.6001,-73.9662,87.94,59,47,7.31,scattered clouds
22,22,Kahului,US,20.8947,-156.4700,78.84,78,20,10.00,few clouds
29,29,Mersing,MY,2.4312,103.8405,78.71,94,20,5.75,few clouds
31,31,Mahon,ES,39.8885,4.2658,75.76,57,0,5.75,clear sky
32,32,Barra Patuca,HN,15.8000,-84.2833,80.98,85,100,15.95,overcast clouds
37,37,Napoleon,US,41.3923,-84.1252,82.76,84,100,3.69,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                197
City                   197
Country                197
Lat                    197
Lng                    197
Max Temp               197
Humidity               197
Cloudiness             197
Wind Speed             197
Weather Description    197
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#n/a

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
11,Kapaa,US,75.18,broken clouds,22.0752,-159.3190,
12,Ukiah,US,77.43,clear sky,39.1502,-123.2078,
15,Bengkulu,ID,77.18,overcast clouds,-3.8004,102.2655,
18,Hilo,US,88.29,overcast clouds,19.7297,-155.0900,
20,Albany,US,87.94,scattered clouds,42.6001,-73.9662,
22,Kahului,US,78.84,few clouds,20.8947,-156.4700,
29,Mersing,MY,78.71,few clouds,2.4312,103.8405,
31,Mahon,ES,75.76,clear sky,39.8885,4.2658,
32,Barra Patuca,HN,80.98,overcast clouds,15.8000,-84.2833,
37,Napoleon,US,82.76,overcast clouds,41.3923,-84.1252,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
hotel_df.count()

City                   197
Country                197
Max Temp               197
Weather Description    197
Lat                    197
Lng                    197
Hotel Name             197
dtype: int64

In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.count()

City                   184
Country                184
Max Temp               184
Weather Description    184
Lat                    184
Lng                    184
Hotel Name             184
dtype: int64

In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))